In [1]:
import os
#os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-1.8.0-openjdk-amd64"
os.environ['PYSPARK_SUBMIT_ARGS'] = ' --packages com.datastax.spark:spark-q-connector_2.11:2.3.0 --packages com.datastax.spark:spark-cassandra-connector_2.11:2.3.2 pyspark-shell'

In [2]:
with open('/Kriptosare.class/tmp/trans_part.txt','r') as file:
    trans_part=int(file.read())
with open("/Kriptosare.class/tmp/turno_bloque.txt",'r') as file:
    turno_bloque=int(file.read())
if trans_part>15:
    appname="part"+str(trans_part)[0]+'_'+str(trans_part)[1:]+"+bloque_codigo"+str(turno_bloque)
else:
    appname="part"+str(trans_part)+"+bloque_codigo"+str(turno_bloque)

In [3]:
import findspark
findspark.init('/opt/spark')
from pyspark import SparkContext,SparkConf
sc = SparkContext()
#with cluster
#conf = (SparkConf()
#         .setMaster("spark://10.200.5.39:7077")
#         .set("spark.driver.host","10.200.5.39") 
#         .set("spark.executor.memory","58g")
#         .set('spark.driver.memory', '58G')
#         .set('spark.driver.maxResultSize','4G')
#         .set('spark.rpc.message.maxSize','512')
#         .setAppName(appname))
#sc = SparkContext(conf=conf)

In [4]:
import pyspark
import binascii
from pyspark.sql import SQLContext
from functools import reduce
#import pygraphviz
import pyspark.sql.functions as f
from IPython.display import Image
#from networkx.drawing.nx_pydot import write_dot
import numpy as np
import matplotlib.pyplot as plt
import copy
from pyspark.sql import Row
import math
import pickle
import shutil
import subprocess

sqlContext = SQLContext(sc)

In [ ]:
from IPython.display import HTML, Javascript, display

def initialize():
    display(HTML(
        '''
            <script>
                code_show = false;
                function restart_run_all(){
                    IPython.notebook.kernel.restart();
                    setTimeout(function(){
                        IPython.notebook.execute_all_cells();
                    }, 15000)
                }
            document.getElementById("demo").innerHTML = restart_run_all();
            </script>
            <p id="demo"></p>  

        '''
    ))

In [ ]:
'''from IPython.display import display_html
def initialize():
    display_html("<script>IPython.notebook.kernel.restart()</script>",raw=True)
    sc.stop()
    !sleep 15
    !jupyter nbconvert --execute Heuristics.ipynb --ExecutePreprocessor.timeout=-1'''

'from IPython.display import display_html\ndef initialize():\n    display_html("<script>IPython.notebook.kernel.restart()</script>",raw=True)\n    sc.stop()\n    !sleep 15\n    !jupyter nbconvert --execute Heuristics.ipynb --ExecutePreprocessor.timeout=-1'

In [ ]:
path = "Kriptosare.class/analysis"
pathDir ="transaction_data_convert_complete/"
pathAux="heuristic_auxiliars/"

In [ ]:
def heuristico1_ad_salida(aux):
    aux0=aux.groupby('address_salida').agg(f.min('user')).withColumnRenamed('address_salida','address_salida1').withColumnRenamed('min(user)','user1').sort('user1')
    
    aux0=aux.join(aux0,aux.address_salida==aux0.address_salida1)\
    .selectExpr('user1 as user','address_salida1 as address_salida','tx_hash as tx_hash','output_number as output_number')\
    

    aux1=aux0.filter(f.col('output_number')==1).groupby(f.col('tx_hash')).agg(f.min('user'))\
    .withColumnRenamed('min(user)','user1').withColumnRenamed('tx_hash','tx_hash1')
    
    kk=aux0.join(aux1,f.col('tx_hash')==f.col('tx_hash1'),how='left')

    user_relation1=kk.select('user1','user').filter(f.col('user1')!=f.col('user')).dropDuplicates()
    
    df_H1_salida_part1=kk.filter(f.col('user1').isNull()).selectExpr('address_salida','tx_hash','output_number','user')
    df_H1_salida_part2=kk.filter(f.col('user1').isNotNull())\
    .selectExpr('address_salida','tx_hash','output_number','user1 as user')
    kk=df_H1_salida_part1.union(df_H1_salida_part2).sort('user')

    return kk.select('tx_hash','address_salida','user').dropDuplicates(),user_relation1

'''def heuristico1_ad_salida(aux,part=1):
    if part==1:
        boolean=True
    else:
        boolean=False
    aux0=aux.groupby('address_salida').agg(f.min('user')).withColumnRenamed('address_salida','address_salida1').withColumnRenamed('min(user)','user1').sort('user1')
    
    aux0=aux.join(aux0,aux.address_salida==aux0.address_salida1)\
    .selectExpr('user1 as user','address_salida1 as address_salida','tx_hash as tx_hash','output_number as output_number')\
    .sort('user',ascending=boolean)

    aux1=aux0.filter(f.col('output_number')==1).dropDuplicates(['tx_hash'])\
    .selectExpr('user as user1', 'tx_hash as tx_hash1', 'address_salida as address_salida1','output_number as output_number1')\
    .sort('user1')
    
    kk=aux0.join(aux1,f.col('tx_hash')==f.col('tx_hash1'),how='left')

    user_relation1=kk.select('user1','user').filter(f.col('user1')!=f.col('user')).dropDuplicates()
    
    df_H1_salida_part1=kk.filter(f.col('user1').isNull()).selectExpr('address_salida','tx_hash','output_number','user')
    df_H1_salida_part2=kk.filter(f.col('user1').isNotNull())\
    .selectExpr('address_salida','tx_hash','output_number','user1 as user')
    kk=df_H1_salida_part1.union(df_H1_salida_part2).sort('user')

    return kk.select('tx_hash','address_salida','user').dropDuplicates(),user_relation1'''
'''
def heuristico1_ad_llegada(df_prueba,df_H1_salida):
    df_H1_llegada=df_prueba.drop('address_salida','timestamp').sort('tx_hash')
    return df_H1_llegada.join(df_H1_salida.drop('tx_hash'),df_H1_salida.address_salida==df_H1_llegada.address_llegada, how='left')\
        .drop('address_salida').dropDuplicates().sort('user')'''

def heuristico1_ad_llegada(df_prueba,df_H1_salida):
    df_H1_llegada=df_prueba.dropDuplicates(['address_llegada']).drop('timestamp').sort('tx_hash')
    kk=df_H1_salida.dropDuplicates(['address_salida']).drop('tx_hash')
    return df_H1_llegada.join(kk,kk.address_salida==df_H1_llegada.address_llegada, how='left')\
        .select('address_llegada','user').dropDuplicates(['address_llegada']).filter(f.col('user').isNotNull()).sort('user')

def heuristico2_ad_llegada(df,df_H1_salida,part=1):
    if part==1:
        df_prueba=df
        kk=df_prueba.filter(f.col('output_number')==2).filter(f.col('new_addresses_per_trans')==1)\
            .dropDuplicates(['address_salida'])
    else:
        df_prueba=df.drop('user')
        kk=df_prueba.filter(f.col('user_llegada').isNull()).filter(f.col('output_number')==2).filter(f.col('new_addresses_per_trans')==1)\
            .dropDuplicates(['address_salida'])
    df_prueba1=kk\
    .join(df_H1_salida,(df_H1_salida.address_salida==kk.address_salida)&(df_H1_salida.tx_hash==kk.tx_hash))\
    .selectExpr('address_llegada as address_llegada1','user').dropDuplicates()
    
    return df_prueba1.join(df_prueba,df_prueba.address_llegada==df_prueba1.address_llegada1,how='right')\
        .select('address_llegada','user').filter(f.col('user').isNotNull()).dropDuplicates(['address_llegada'])    
    
def union_heur_ad_llegada(df_H1_llegada,df_H2_llegada):
    df_H1_aux=df_H1_llegada.selectExpr('address_llegada as address_llegada1', 'user as user1')
    return df_H2_llegada\
        .join(df_H1_aux,df_H2_llegada.address_llegada==df_H1_aux.address_llegada1)\
        .selectExpr('address_llegada1 as address_llegada', 'user1 as user1', 'user as user').dropDuplicates().sort('user')

        
def user_per_transaction(df_prueba,part=1):      
        #ponemos un usuario por cada transacción
        if part==1:
            aux=df_prueba.filter(df_prueba.address_salida.isNotNull()).drop('address_llegada').sort('timestamp','tx_hash')
        else:
            aux=df_prueba.filter(df_prueba.address_salida.isNotNull()).sort('timestamp','tx_hash')
        aux=aux.collect()
        with open("/Kriptosare.class/tmp/cont.txt",'r') as file:
            cont=int(file.read())
        for i in range(len(aux)-1):
            rdict=aux[i].asDict()
            rdict['user']="user"+'0'*(13-len(str(cont)))+str(cont)
            aux[i]=Row(**rdict)
            if aux[i]['tx_hash']!=aux[i+1]['tx_hash'] or (aux[i]['output_number']!=1):
                cont=cont+1
        rdict=aux[len(aux)-1].asDict()
        rdict['user']="user"+'0'*(13-len(str(cont)))+str(cont)
        aux[len(aux)-1]=Row(**rdict)
        aux=sqlContext.createDataFrame(aux)
        
        with open("/Kriptosare.class/tmp/cont.txt",'w') as file:
            file.write(str(cont+1))
        if part==1:
            return aux.drop('timestamp').sort('user')
        else:
            return aux.sort('user')
        
def cluster_add_with_user(df_H1_salida,df_H_llegada,user_representation):
    df=df_H1_salida.selectExpr('address_salida as address','user')\
    .union(df_H_llegada.selectExpr('address_llegada as address','user')).dropDuplicates(['address'])
    
    df=df.join(user_representation,user_representation.user1==df.user,how='left')
    df1=df.filter(f.col('representator').isNotNull()).select('address','representator')
    df2=df.filter(f.col('representator').isNull()).selectExpr('address','user as representator')
    return df1.union(df2).withColumnRenamed('representator','user').dropDuplicates()

        
def ad_restantes(df_prueba,df_H_cluster):
    add_totales=df_prueba.selectExpr('address_salida as address1').union(df_prueba.select('address_llegada'))\
        .filter(f.col('address1').isNotNull()).dropDuplicates()
    
    return add_totales.join(df_H_cluster,f.col('address1')==f.col('address'),how='left')\
        .filter(f.col('address').isNull()).selectExpr('address1 as address').dropDuplicates()      
    
    
    
    
    
"""def ad_restantes(df_prueba):
    with open("/home/hadoop/Heuristics/cont.txt",'r') as file:
        cont=int(file.read())
    
    aux2=df_prueba.filter(df_prueba.address_salida.isNull()).drop('address_salida').dropDuplicates(['address_llegada'])
    aux2=aux2.collect()
    for i in range(len(aux2)):
        cont=cont+1
        rdict=aux2[i].asDict()
        rdict['user']="user"+'0'*(10-len(str(cont)))+str(cont)
        aux2[i]=Row(**rdict)
        
    with open("/home/hadoop/Heuristics/cont.txt",'w') as file:
        file.write(str(cont+1))
    
    aux2=sqlContext.createDataFrame(aux2)
    return aux2.selectExpr('address_llegada as address', 'user as user')
"""
def ad_restantes_raw(address):
    with open("/Kriptosare.class/tmp/cont.txt",'r') as file:
        cont=int(file.read())
    aux2=address.collect()
    for i in range(len(aux2)):
        cont=cont+1
        rdict=aux2[i].asDict()
        rdict['user']="user"+'0'*(13-len(str(cont)))+str(cont)
        aux2[i]=Row(**rdict)
    with open("/Kriptosare.class/tmp/cont.txt",'w') as file:
        file.write(str(cont+1))
    
    aux2=sqlContext.createDataFrame(aux2)
    return aux2
        
def user_repr(df_H_llegada=True,add_user_rel=True):
    #Pequeño cambio para combinar df_H_llegada y user_relation para crear otro user_relation
    if add_user_rel==True:
        user_relation=df_H_llegada.drop('address_llegada').dropDuplicates().sort('user')
    else:
        if df_H_llegada!=True:
            user_relation=df_H_llegada.drop('address_llegada').union(add_user_rel).dropDuplicates().sort('user')
        else:
            user_relation=add_user_rel
    urel_list=user_relation.filter(f.col('user').isNotNull()).filter(f.col('user1').isNotNull()).dropDuplicates().collect()
#Cambios aquí
    urel_list1=[]
    h={}
    rep_list=[]
    for i in range(len(urel_list)):
        rdict=urel_list[i].asDict()        
        if rdict['user1']!=rdict['user']:             
            if (rdict['user1'] not in h.keys()) and (rdict['user'] not in h.keys()):
                h[rdict['user']]={rdict['user1'],rdict['user']}
                h[rdict['user1']]=rdict['user']
                rep_list.append(rdict['user'])

            elif (rdict['user1'] in h.keys()) and (rdict['user'] not in h.keys()):

                u1=rel_degree(h,rdict['user1'])
                h[u1].add(rdict['user'])
                h[rdict['user']]=u1  


            elif (rdict['user'] in h.keys()) and (rdict['user1'] not in h.keys()):

                u=rel_degree(h,rdict['user'])
                h[u].add(rdict['user1'])
                h[rdict['user1']]=u

            else:
                u1=rel_degree(h,rdict['user1'])
                u=rel_degree(h,rdict['user'])
                if u1==u:
                    pass
                else:
                    if len(h[u1])>len(h[u]):
                        h[u1]=h[u1].union(h[u])
                        h[u]=u1
                        rep_list.remove(u)
                    else:
                        h[u]=h[u].union(h[u1])
                        h[u1]=u
                        rep_list.remove(u1)
    for i in rep_list:
        for j in h[i]:        
            urel_list1.append(Row(representator=i,user1=j))
    return sqlContext.createDataFrame(urel_list1)

def rel_degree(h,key):
    a=key
    l=[a]
    c=0
    if type(h[a])==type(set()):
        return a
    while type(h[a])!=type(set()):       
        c+=1
        a=h[a]
        if (a not in l):
            l.append(a)
        else:
            print('hay un bucle',a,c)
            return
    return a

def column_add(df_prueba,part=1):
    df1=df_prueba.groupby(f.col('tx_hash')).agg(f.countDistinct('address_llegada'))\
    .withColumnRenamed('tx_hash','tx_hash1').withColumnRenamed('count(DISTINCT address_llegada)','output_number')
    df1=df_prueba.join(df1,df_prueba.tx_hash==df1.tx_hash1,how='left').drop('tx_hash1').sort('timestamp')
    if part==1:
        df2=df1.dropDuplicates(['address_llegada']).groupby(f.col('tx_hash')).agg(f.countDistinct('address_llegada'))\
        .withColumnRenamed('count(DISTINCT address_llegada)','new_addresses_per_trans').withColumnRenamed('tx_hash','tx_hash1')
    else:
        df2=df1.filter(f.col('user_llegada').isNull()).dropDuplicates(['address_llegada']).groupby(f.col('tx_hash')).agg(f.countDistinct('address_llegada'))\
        .withColumnRenamed('count(DISTINCT address_llegada)','new_addresses_per_trans').withColumnRenamed('tx_hash','tx_hash1')\
        
    return df1.join(df2,df1.tx_hash==df2.tx_hash1,how='left')\
        .withColumn("new_addresses_per_trans", f.when(f.col("new_addresses_per_trans").isNull(), 0)\
        .otherwise(f.col("new_addresses_per_trans"))).drop('tx_hash1')
    
def delete_file(some_path):
    subprocess.call(["hdfs", "dfs", "-rm", "-r", some_path])
def rename_file(some_path,name):
    subprocess.call(["hdfs", "dfs", "-mv", some_path, name])
    
def spliting_func(df_no_user):
    l=sqlContext.read.parquet(path+pathDir+"transaction_data_convert_part"+str(trans_part)+".parquet")\
        .select('timestamp').dropDuplicates().sort('timestamp').collect()
    min_val=l[0][0]
    max_val=l[len(l)-1][0]
    N=df_no_user.count()
    n=math.floor(N/21343813)
    df_no_users=[]
    k=min_val
    delta_x=max_val-min_val
    for i in range(n):
        if i<n-1:
            df_no_users.append(df_no_user.filter((f.col('timestamp')>=k)&(f.col('timestamp')<k+(delta_x/2))))
            k+=(delta_x/2)
            delta_x=delta_x/2
            
        else:
            df_no_users.append(df_no_user.filter(f.col('timestamp')>=k))
    
    big_aux=user_per_transaction(df_no_users[0].sort('timestamp','tx_hash'),0)
    
    for i in range(1,len(df_no_users)):
        big_aux=big_aux.union(user_per_transaction(df_no_users[i].sort('timestamp','tx_hash'),0))
    return big_aux.dropDuplicates().sort('user')

def spliting_add_raw(addresses):
    n=math.floor(addresses.count()/21343813)
    df=addresses.withColumn('counter',f.monotonically_increasing_id())
    max_val=df.agg(f.max('counter')).collect()[0][0]
    
    min_val=df.agg(f.min('counter')).collect()[0][0]
    address_split=[]
    for i in range(n):
        if i<n-1:
            address_split.append(df.filter((f.col('counter')>=min_val+(max_val-min_val)*i/n)&(f.col('counter')<min_val+(max_val-min_val)*(i+1)/n)))
        else:
            address_split.append(df.filter((f.col('counter')>=min_val+(max_val-min_val)*i/n)&(f.col('counter')<=min_val+(max_val-min_val)*(i+1)/n)))
    big_aux2=ad_restantes_raw(address_split[0])       
    for i in range(1,len(address_split)):
        big_aux2=big_aux2.union(ad_restantes_raw(address_split[i]))
    return big_aux2.drop('counter')

In [ ]:
def heuristic_clustering_no_restrictions():
        df=sqlContext.read.parquet(path+pathDir+"transaction_data_convert_part1.parquet")
        df_prueba=column_add(df)
            
        #ponemos un usuario por cada transacción
        with open("/Kriptosare.class/tmp/checkpoint.txt",'w') as file:
                file.write('user_per_transaction')
        aux=user_per_transaction(df_prueba)
        
        
        #pasamos heurístico 1 a los address_salida
        with open("/Kriptosare.class/tmp/checkpoint.txt",'w') as file:
                file.write('heuristico1_ad_salida')
        df_H1_salida,user_relation=heuristico1_ad_salida(aux)  
        
        #hacemos que los address_llegada que están en la salida de alguna transacción reciban el user(hereden):
        with open("/Kriptosare.class/tmp/checkpoint.txt",'w') as file:
                file.write('heuristico1_ad_llegada')
        df_H1_llegada=heuristico1_ad_llegada(df_prueba,df_H1_salida)
        
        #Pasamos el heurístico 2 en address_llegada:
        with open("/Kriptosare.class/tmp/checkpoint.txt",'w') as file:
                file.write('heuristico2_ad_llegada')
        df_H2_llegada=heuristico2_ad_llegada(df_prueba,df_H1_salida)
        
        
        #Unimos los heuristicos en address_llegada para sacar relaciones:
        with open("/Kriptosare.class/tmp/checkpoint.txt",'w') as file:
                file.write('union_heur_ad_llegada')
        df_H_llegada=union_heur_ad_llegada(df_H1_llegada,df_H2_llegada)
        
        
        #Sacamos la relacion de users para así obtener un único representante para cada user:
        with open("/Kriptosare.class/tmp/checkpoint.txt",'w') as file:
                file.write('user_repr')
        user_representation=user_repr(df_H_llegada,user_relation)    
        
        
        #Con las relaciones obtenidas hacemos un cluster para las address_salida
        with open("/Kriptosare.class/tmp/checkpoint.txt",'w') as file:
                file.write('cluster_add_with_user')
                
        df_H_cluster=cluster_add_with_user(df_H1_salida,df_H_llegada,user_representation)
        
        #Unimos ambos clusters y lo guardamos     
        with open("/Kriptosare.class/tmp/checkpoint.txt",'w') as file:
                file.write('ad_restantes')
                
        addresses=ad_restantes(df_prueba,df_H_cluster) 
        
        with open(path+"txt_files\\checkpoint.txt",'w') as file:
                file.write('ad_restantes_raw')
                
        aux2=ad_restantes_raw(addresses)
        
        with open("/Kriptosare.class/tmp/checkpoint.txt",'w') as file:
                file.write('save')
                
        df_H_cluster=df_H_cluster.union(aux2).sort('user').dropDuplicates(['address'])
        
        df_H_cluster.write\
                    .format("org.apache.spark.sql.cassandra")\
                    .mode('append')\
                    .options(table="cluster", keyspace="kryptosare")\
                    .save()

        
#def heuristic_clustering_no_restrictions_partial_process(df_prueba,df_H1_salida):
        
        
        #hacemos que los address_llegada que están en la salida de alguna transacción reciban el user(hereden):
#        df_H1_llegada=heuristico1_ad_llegada(df_prueba,df_H1_salida)

        #Pasamos el heurístico 2 en address_llegada:
#        df_H2_llegada=heuristico2_ad_llegada(df_prueba,df_H1_salida)

        
        #Unimos los heuristicos en address_llegada para sacar relaciones:
#        df_H_llegada=union_heur_ad_llegada(df_H1_llegada,df_H2_llegada)

        #Si la parte no es la primera sólo necesitamos el user_relation y address_salida con sus user
        #user_relation=df_H_llegada.drop('address_llegada').dropDuplicates().sort('user')
#        return df_H_llegada.drop('address_llegada').dropDuplicates().sort('user'),df_H2_llegada

In [ ]:
def clustering():
    
    if trans_part==1:
        with open("/Kriptosare.class/tmp/app_name.txt",'w') as file:
                file.write('part'+str(trans_part)+'+'+'bloque_código'+str(turno_bloque))
        heuristic_clustering_no_restrictions()
        with open("/Kriptosare.class/tmp/trans_part.txt",'w') as file:
            file.write(str(trans_part+1))
    else:
        with open("/Kriptosare.class/tmp/app_name.txt",'w') as file:
                file.write('part'+str(trans_part)+'+'+'bloque_código'+str(turno_bloque))
        eval('bloque'+str(turno_bloque)+'()') 
        if trans_part==55 and turno_bloque==9:
            with open("/Kriptosare.class/tmp/checkpoint.txt",'w') as file:
                file.write('finished')
            os.__exit(00)
            return
        
        if turno_bloque<9:
            with open("/Kriptosare.class/tmp/turno_bloque.txt",'w') as file:
                file.write(str(turno_bloque+1))
        else:
            with open("/Kriptosare.class/tmp/turno_bloque.txt",'w') as file:
                file.write('1')
                
    initialize()   
        

        

def bloque1():        
        #joins:2
        with open("/Kriptosare.class/tmp/checkpoint.txt",'w') as file:
                file.write('running')
        df_prueba=sqlContext.read.parquet(path+pathDir+"transaction_data_convert_part"+str(trans_part)+".parquet").dropDuplicates()
        df_cluster_part=sqlContext.read\
                .format("org.apache.spark.sql.cassandra")\
                .options(table='cluster', keyspace='kryptosare')\
                .load()\
                .withColumnRenamed('address','address1')
        #df_cluster_part.show()
            
        df_prueba_step=df_prueba.join(df_cluster_part,df_cluster_part.address1==df_prueba.address_salida,how='left')\
            .drop('address1').withColumnRenamed('user','user1')
    
            
        df_prueba_step=df_prueba_step\
        .join(df_cluster_part,df_cluster_part.address1==df_prueba_step.address_llegada,how='left')\
        .drop('address1').withColumnRenamed('user','user_llegada').withColumnRenamed('user1','user')
        
        column_add(df_prueba_step,0).write.parquet(path+pathAux+'df_prueba_step.parquet',mode='overwrite')
        
def bloque2():
        df_prueba_step=sqlContext.read.parquet(path+pathAux+'df_prueba_step.parquet').dropDuplicates()
    
        df_no_miner=df_prueba_step.filter(f.col('address_salida').isNotNull()).sort('timestamp','user',ascending=[True,False])

        #pasamos el H1 a la tabla con los user actuales para obtener posibles relaciones y añadir users en una misma transacción
        
        df_no_user=df_no_miner.filter(f.col('user').isNull()).drop('user')
        
        aux=user_per_transaction(df_no_user,0)\
        .select('address_salida','tx_hash','address_llegada','timestamp','amount','output_number','new_addresses_per_trans','user','user_llegada')
        
        aux=aux.union(df_no_miner.filter(f.col('user').isNotNull()))
        
        aux.write.parquet(path+pathAux+'aux.parquet',mode='overwrite')
        df_no_miner.write.parquet(path+pathAux+'df_no_miner.parquet',mode='overwrite')

        
def bloque3():
        aux=sqlContext.read.parquet(path+pathAux+'aux.parquet')
        df_no_miner=sqlContext.read.parquet(path+pathAux+'df_no_miner.parquet')

        
        df_no_miner=df_no_miner.filter(f.col('user').isNotNull()).union(aux)
        
        df_H1_salida,user_relation=heuristico1_ad_salida(df_no_miner)
        
        
        
        df_no_miner.write.parquet(path+pathAux+'apoyo1.parquet',mode='overwrite')
        user_relation.write.parquet(path+pathAux+'user_relation.parquet',mode='overwrite')
        df_H1_salida.write.parquet(path+pathAux+'df_H1_salida.parquet',mode='overwrite')       
        
def bloque4():      
    df_no_miner=sqlContext.read.parquet(path+pathAux+'apoyo1.parquet') 
    df_H1_salida=sqlContext.read.parquet(path+pathAux+'df_H1_salida.parquet')
    
    df_H1_llegada=heuristico1_ad_llegada(df_no_miner.drop('user'),df_H1_salida.drop('output_number'))
    
    df_H1_llegada.write.parquet(path+pathAux+'df_H1_llegada.parquet',mode='overwrite')
    
def bloque5():
    df_no_miner=sqlContext.read.parquet(path+pathAux+'apoyo1.parquet')
    df_H1_salida=sqlContext.read.parquet(path+pathAux+'df_H1_salida.parquet')
    
    df_H2_llegada=heuristico2_ad_llegada(df_no_miner,df_H1_salida,part=0)
        
    df_H2_llegada.write.parquet(path+pathAux+'df_H2_llegada.parquet',mode='overwrite')   

    
def bloque6():
    df_H1_llegada=sqlContext.read.parquet(path+pathAux+'df_H1_llegada.parquet')
    df_H2_llegada=sqlContext.read.parquet(path+pathAux+'df_H2_llegada.parquet')
    
    df_H_llegada=union_heur_ad_llegada(df_H1_llegada,df_H2_llegada)
    
    df_H_llegada.write.parquet(path+pathAux+'df_H_llegada.parquet',mode='overwrite')
    
def bloque7():
    df_H1_salida=sqlContext.read.parquet(path+pathAux+'df_H1_salida.parquet')
    df_H_llegada=sqlContext.read.parquet(path+pathAux+'df_H_llegada.parquet')
    user_relation=sqlContext.read.parquet(path+pathAux+'user_relation.parquet')

    user_representation=user_repr(df_H_llegada,user_relation)  
        
                
    df_H_cluster=cluster_add_with_user(df_H1_salida,df_H_llegada,user_representation)
    
    user_representation.write.parquet(path+pathAux+'user_representation.parquet',mode='overwrite')
    df_H_cluster.write.parquet(path+pathAux+'df_H_cluster.parquet',mode='overwrite')
        
def bloque8():                
    df_prueba=sqlContext.read.parquet(path+pathDir+"transaction_data_convert_part"+str(trans_part)+".parquet").dropDuplicates()
    df_H_cluster=sqlContext.read.parquet(path+pathAux+'df_H_cluster.parquet')
        
    addresses=ad_restantes(df_prueba,df_H_cluster)
    aux2=ad_restantes_raw(addresses)

    
    aux2.write.parquet(path+pathAux+'aux2.parquet',mode='overwrite')
    
def bloque9():
    user_representation=sqlContext.read.parquet(path+pathAux+'user_representation.parquet').withColumnRenamed('user','user1')
    aux2=sqlContext.read.parquet(path+pathAux+'aux2.parquet').withColumnRenamed('user','user1')
    df_H_cluster=sqlContext.read.parquet(path+pathAux+'df_H_cluster.parquet')
    old_cluster=sqlContext.read\
                .format("org.apache.spark.sql.cassandra")\
                .options(table='cluster', keyspace='kryptosare')\
                .load()
    
    new_cluster=df_H_cluster.union(aux2).sort('user').dropDuplicates(['address'])
    
    
    #df_cluster_part=old_cluster.join(user_representation,user_representation.user1==old_cluster.user,how='left')
    #old_cluster_no_update=df_cluster_part.filter(f.col('representator').isNull()).select('address','user')
    old_cluster_update=old_cluster.join(user_representation,user_representation.user1==old_cluster.user).selectExpr('address','representator as user')
    
    
    old_cluster_update.union(new_cluster).dropDuplicates(['address'])\
    .write.parquet(path+pathAux+'cluster_apoyo.parquet',mode='overwrite')
    cluster=sqlContext.read.parquet(path+pathAux+'cluster_apoyo.parquet')
    
    cluster\
                    .write\
                    .format("org.apache.spark.sql.cassandra")\
                    .mode('append')\
                    .options(table="cluster", keyspace="kryptosare")\
                    .save()
    
    with open('/Kriptosare.class/tmp/trans_part.txt','w') as file:
                file.write(str(trans_part+1))

In [ ]:
clustering()